# Implementación de word2vec

El objetivo del algoritmo `word2vec` es encontrar la representación de una palabra en un espacio vectorial a partir de los diferentes contextos en que aparece dicha palabra. Además, la intuición dice que si puedo predecir correctamente la palabra a partir de un contexto o el contexto a partir de la palabra, entonces conozco su significado correctamente.

A partir de lo anterior vamos a definir un modelo neuronal para estimar la probabilidad $P(O=o|C=c)$, que representa la probabilidad de que, dada una palabra central $c$, se obtenga en su contexto la palabra $o$ de un vocabulario $V = \{ w_1, \ldots, w_c, \ldots, w_o, \ldots, w_{|V|}\}$. Esta probabilidad se calcula como la salida de una softmax después de haber pasado por un layer $V$ de entrada y uno $U$ de salida:

$$
P(o|c) = P(u_o|v_c) = \frac{e^{u_o^Tv_c}}{\sum_{k=1}^{|V|} e^{u_k^Tv_c}}
$$

donde $u_k$ y $v_k$ son la $k$-ésima fila de las matrices $U$ y $V$ respectivamente.

Las muestras para entrenar este modelo se componen de un conjunto de palabras y sus respectivos contextos $\{x^{(i)},\{y^{(i,1)},\ldots,y^{(i,2n)} \} \}_{i=1}^N$, tomados de un corpus de entrenamiento. Además, la función de costo tomada para el entrenamiento es la que se obtiene por *maximum likelihood*, asumiendo independencia de todas las muestras:

$$
\begin{align}
L &= -\log\left(\prod_{i=1}^N \prod_{j=1}^{2n} P(u_{y^{(i,j)}}|v_{x^{(i)}})\right) \\[.5em]
&= -\sum_{i=1}^N \sum_{j=1}^{2n} \log\left(P(u_{y^{(i,j)}}|v_{x^{(i)}})\right) \\[.5em]
&= -\sum_{i=1}^N \sum_{j=1}^{2n} \sum_{k=1}^{|V|} \mathbb{1}_{\left\{y^{(i,j)} = k\right\}}\log\left(P(u_{k}|v_{x^{(i)}}\right) \\[.5em]
&= \sum_{i=1}^N \sum_{j=1}^{2n} CE \left( P(u|v_{x^{(i)}}),\hat{y}^{(i,j)} \right)\\[.5em]
\end{align}
$$

donde $\hat{y}^{(i,j)} \in \mathbb{R}^{|V|}$ es un vector con la $y^{(i,j)}$-ésima componente igual a 1 y cero en el resto.


Para el cálculo del gradiente de la función de costo necesitamos calcular las siguientes derivadas parciales:

$$
\frac{\partial}{\partial v_i}\log P(u_o|v_c) \;\; \forall i=1,\ldots,|V| \\
\frac{\partial}{\partial u_j}\log P(u_o|v_c) \;\; \forall j=1,\ldots,|V| \\
$$

Empecemos:

$$
\begin{align}
\frac{\partial}{\partial v_i}\log P(u_o|v_c) &= \left(\frac{\partial}{\partial v_i} \log P(u_o|v_i) \right) \mathbb{1}_{\left\{ i = c \right\}} \\[1em]
&= \left(\frac{\partial}{\partial v_i} u_o^Tv_i - \log\left( \sum_w e^{u_w^T v_i} \right)\right) \mathbb{1}_{\left\{ i = c \right\}} \\[1em]
&= \left( u_o - \frac{\partial}{\partial v_i} \log\left( \sum_w e^{u_w^T v_i} \right)\right) \mathbb{1}_{\left\{ i = c \right\}} \\[1em]
&= \left( u_o - \frac{1}{\sum_x e^{u_x^T v_i} } \frac{\partial}{\partial v_i} \sum_w e^{u_w^T v_i} \right) \mathbb{1}_{\left\{ i = c \right\}} \\[1em]
&= \left( u_o - \frac{\sum_w u_w e^{u_w^T v_i}}{\sum_x e^{u_x^T v_i} } \right) \mathbb{1}_{\left\{ i = c \right\}} \\[1em]
&= \left(\sum_w u_w \mathbb{1}_{\left\{ w = o \right\}} - \sum_w u_w \frac{e^{u_w^T v_i}}{\sum_x e^{u_x^T v_i} } \right) \mathbb{1}_{\left\{ i = c \right\}} \\[1em]
&= \left(\sum_w u_w \left(\hat{y}_w - P(u_w|v_i)\right) \right) \mathbb{1}_{\left\{ i = c \right\}} \\[1em]
\frac{\partial}{\partial V} P(u_o|v_c) &= U \left(y - \hat{y} \right)
\end{align}
$$



$$
\begin{align}
\frac{\partial}{\partial u_j}\log P(u_o|v_c) &= \frac{\partial}{\partial u_j} u_o^Tv_c - \frac{\partial}{\partial u_j} \log\left( \sum_w e^{u_w^T v_c} \right) \\[1em]
&= v_c \mathbb{1}_{\left\{ j = o \right\}} - \frac{\sum_w \frac{\partial}{\partial u_j} e^{u_w^T v_c}}{\sum_x e^{u_x^T v_c} } \\[1em]
&= v_c \mathbb{1}_{\left\{ j = o \right\}} - \frac{\sum_w v_c e^{u_w^T v_c} \mathbb{1}_{\left\{ w = j \right\}} }{\sum_x e^{u_x^T v_c} } \\[1em]
&= (\mathbb{1}_{\left\{ j = o \right\}} - \frac{e^{u_j^T v_c} }{\sum_x e^{u_x^T v_c} } ) v_c   \\[1em]
&= (\hat{y}_j - y_j) v_c   \\[1em]
\frac{\partial}{\partial U} P(u_o|v_c) &= V \left(y - \hat{y} \right) \\[1em]
\end{align}
$$

Necesitamos la derivada de $\sigma(x)$ para hacer Negative Sampling:

$$
\sigma ' (x) = \sigma(x) (1-\sigma(x))
$$

In [21]:
a = np.array([[1,2,3]]).T
b = np.arange(9).reshape(3,3)
b.sum(axis=0,keepdims=True)
b

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])